## 1. 填写账号密码



In [ ]:
import requests

def login():
    
    username = ""
    password = ""
    s = requests.Session()
    s.auth = (username, password)
    response = s.post('https://api.worldquantbrain.com/authentication')
    print(response.content)
    return s 

## 2.一些函数的定义

In [ ]:
from os import environ
from time import sleep
import time
import json
import pandas as pd
import random
import pickle
from itertools import product
from itertools import combinations
from collections import defaultdict
from urllib.parse import urljoin
import pickle
import requests
import json
from os.path import expanduser
from requests.auth import HTTPBasicAuth
from datetime import datetime, timezone, timedelta
def get_latest_monday(tz=None):
    if tz is None:
        tz = timezone(timedelta(hours=-4))
    
    now = datetime.now(tz)
    days_since_monday = now.weekday()
    monday = now - timedelta(days=days_since_monday)
    monday_start = monday.replace(hour=0, minute=0, second=0, microsecond=0)
    return monday_start.isoformat(timespec='minutes')

reverse_super_alpha_list = []
def super_score(rec):
    # 一个粗略计算super_alpha得分的函数,权重可调整
    sharpe = rec[1]**3
    turnover = rec[2]
    fitness = rec[3]**3
    margin = rec[4]
    if(rec[5]==None):
        pc=9999999
    else:
        pc=rec[6]*1000000
    return sharpe * (1 - turnover) *fitness *  margin -pc

def check_prod(alpha_bag,start):
    s = login()
    for idx, i in enumerate(alpha_bag):
        if idx < start:
            continue
        if idx % 100 == 0:
            print(idx)
        if idx % 200 == 0 & idx!=0 :
            s = login()
        #print(idx)
        pc = get_check_prod(s, i[0])
        i.append(pc)
        alpha_bag[idx] = i
        print(i)
        print(f"Processed {idx} alphas")


def get_check_prod(s, alpha_id):
    while True:
        result = s.get(f"https://api.worldquantbrain.com/alphas/{alpha_id}/correlations/prod")
        if "retry-after" in result.headers:
            time.sleep(float(result.headers["Retry-After"]))
        elif result.status_code == 429:
            print("Rate limit exceeded, sleeping for 60 seconds")
            time.sleep(60)
            continue
        else:
                try:
                    r = result.json()
                    print({alpha_id: r['max']})
                    return r['max']
                except Exception as e:
                    print(f"Error for alpha_id: {alpha_id}, {e}")
                return

def score(rec):
    # 一个粗略计算alpha得分的函数,权重可调整
    sharpe = rec[1]*5
    turnover = rec[2]
    fitness = rec[3]*5
    margin = rec[4]*2
    return sharpe * (1 - turnover) *fitness *  margin 
def verfiy_result(res):
    checks = res['is']['checks']
    flag = True
    for check in checks:
        if check['result'] in ['FAIL', 'ERROR']:
            flag = False
            if(check['name'] == "LOW_SHARPE"):
                reverse_super_alpha_list.append([res['id'],res['is']['sharpe'],res['is']['fitness'],res['is']['margin']])
            print("IS failed:", res['id'] , check['name'], check['result'])
            break
    return flag
def sus(start, end, sharpe_th, fitness_th,alpha_num, usage):
    s = login()
    output = []
    # 3E large 3C less
    count = 0
    for i in range(0, alpha_num, 100):
        url_e = "https://api.worldquantbrain.com/users/self/alphas?limit=100&offset=%d"%(i) \
                + "&status=UNSUBMITTED%1FIS_FAIL&dateCreated%3E=" + start  \
                + "&dateCreated%3C" + end \
                + "&is.fitness%3E" + str(fitness_th) + "&is.sharpe%3E" \
                + str(sharpe_th) + "&order=-is.sharpe&hidden=false&type=SUPER"
        url_c = "https://api.worldquantbrain.com/users/self/alphas?limit=100&offset=%d"%(i) \
                + "&status=UNSUBMITTED%1FIS_FAIL&dateCreated%3E=" + start  \
                + "&dateCreated%3C" + end \
                + "&is.fitness%3C-" + str(fitness_th) + "&is.sharpe%3C-" \
                + str(sharpe_th) +  "&order=is.sharpe&hidden=false&type=SUPER"
        urls = [url_e]
        if usage != "submit":
            urls.append(url_c)
        for url in urls:
            response = s.get(url)
            #print(response.json())
            try:
                alpha_list = response.json()["results"]
                #print(response.json())
                
                for j in range(len(alpha_list)):
                    if not verfiy_result(alpha_list[j]):
                        continue
                    alpha_id = alpha_list[j]["id"]
                    name = alpha_list[j]["name"]
                    dateCreated = alpha_list[j]["dateCreated"]
                    sharpe = alpha_list[j]["is"]["sharpe"]
                    fitness = alpha_list[j]["is"]["fitness"]
                    turnover = alpha_list[j]["is"]["turnover"]
                    margin = alpha_list[j]["is"]["margin"]
                    longCount = alpha_list[j]["is"]["longCount"]
                    shortCount = alpha_list[j]["is"]["shortCount"]
                    decay = alpha_list[j]["settings"]["decay"]
                    count += 1
                    #if (sharpe > 1.2 and sharpe < 1.6) or (sharpe < -1.2 and sharpe > -1.6):
                    if (longCount + shortCount) > 100:
                        rec = [alpha_id,sharpe, turnover, fitness, margin, decay]
                        print(rec)
                        if turnover > 0.7:
                            rec.append(decay*4)
                        elif turnover > 0.6:
                            rec.append(decay*3+3)
                        elif turnover > 0.5:
                            rec.append(decay*3)
                        elif turnover > 0.4:
                            rec.append(decay*2)
                        elif turnover > 0.35:
                            rec.append(decay+4)
                        elif turnover > 0.3:
                            rec.append(decay+2)
                        output.append(rec)
                        
            except Exception as e:
                print(f"Error in response for url: {url}")
                print(f"Exception type: {type(e)}")
                error_check=response
                print(f"{i} finished re-login")
                s = login()
    print("count: %d"%count)
    output.sort(key=score, reverse=True)
    return output  

         

## 3.筛选本周super alpha

- 通过datatime函数 获取本周开始时间与现在时间作为alpha筛选范围(每周不同alpha副本)
- 便于修改的筛选条件
- 自动过滤IS failed alpha 并给出failed:原因
- 对于shapre,fitness绝对值大小符合筛选条件,符号为负的alpha,单独列出,以待修改


In [ ]:

tz_ny = timezone(timedelta(hours=-4))
start = get_latest_monday(tz_ny) 
end = datetime.now(tz_ny).isoformat(timespec='minutes')

print(f"开始时间(默认本周一开始时间): {start}")
print(f"结束时间: {end}")

开始时间(默认本周一开始时间): 2025-06-16T00:00-04:00
结束时间: 2025-06-18T13:09-04:00


In [ ]:
'''
起始时间 结束时间 shapre fitness
对于开始时间想要细调,也可以换成
super_list=sus("2025-06-17T06:06-04:00",end,5,5,300, "track")
这样的形式 来细调
避免反复check 某些alpha的prod corr
'''
super_list=sus(start,end,5,5,300, "track")
super_list

b'{"user":{"id":"WY79548"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
['E88w0oR', 8.41, 0.1448, 9.44, 0.002521, 3]
['d3vj2OJ', 8.36, 0.1472, 9.32, 0.002483, 3]
['qYLv6MA', 8.22, 0.135, 9.49, 0.002666, 3]
['QNgM7Pw', 7.84, 0.1487, 8.46, 0.002327, 3]
['NJNaeQw', 7.61, 0.1327, 9.5, 0.003117, 3]
['w2Weav2', 7.6, 0.1395, 9.36, 0.003034, 3]
['RlzGG6j', 7.6, 0.1395, 9.36, 0.003034, 3]
['JW1VaLx', 7.54, 0.1396, 9.64, 0.003272, 3]
['NJeaoAe', 7.39, 0.1116, 10.34, 0.004381, 3]
['m0a2grE', 7.36, 0.1273, 9.97, 0.003672, 3]
['kWOE8aK', 7.35, 0.156, 8.44, 0.002638, 3]
['qYGQAQ1', 7.35, 0.1519, 8.88, 0.00292, 3]
['7vvxpG1', 7.35, 0.156, 8.44, 0.002638, 3]
['9ddAdRo', 7.35, 0.1519, 8.88, 0.00292, 3]
['pMxmdKX', 7.35, 0.1519, 8.88, 0.00292, 3]
['XvMEV8z', 7.29, 0.1169, 8.04, 0.002604, 3]
['pMxeldX', 7.29, 0.1465, 8.19, 0.002527, 3]
['

[['NJeaoAe', 7.39, 0.1116, 10.34, 0.004381, 3],
 ['m0a2grE', 7.36, 0.1273, 9.97, 0.003672, 3],
 ['2X5xvj5', 7.1, 0.0835, 8.45, 0.004242, 3],
 ['ggeGlgJ', 7.0, 0.0835, 8.35, 0.004259, 3],
 ['6od52QK', 5.59, 0.0689, 7.11, 0.005883, 3],
 ['br6EkqZ', 5.59, 0.0689, 7.11, 0.005883, 3],
 ['aQGRdl9', 6.93, 0.0835, 8.19, 0.004179, 3],
 ['qYGRlpE', 6.89, 0.0835, 8.16, 0.004201, 3],
 ['Lmm13LM', 6.82, 0.0835, 8.12, 0.004248, 3],
 ['1lOMwVm', 6.82, 0.0835, 8.12, 0.004248, 3],
 ['OMAgj0d', 6.01, 0.071, 7.25, 0.005124, 3],
 ['JW1VaLx', 7.54, 0.1396, 9.64, 0.003272, 3],
 ['l5g8zON', 5.17, 0.0689, 6.76, 0.006194, 3],
 ['E8QPGXJ', 5.17, 0.0689, 6.76, 0.006194, 3],
 ['RlAbbda', 6.75, 0.1293, 9.13, 0.003659, 3],
 ['NJNaeQw', 7.61, 0.1327, 9.5, 0.003117, 3],
 ['QNN9k0M', 5.1, 0.0667, 6.58, 0.006238, 3],
 ['6oGl5N7', 7.01, 0.0842, 7.95, 0.003819, 3],
 ['XvRJkV5', 6.82, 0.1294, 9.14, 0.003591, 3],
 ['3kPNWYQ', 5.1, 0.0669, 6.58, 0.006223, 3],
 ['NJzdJrg', 5.02, 0.0668, 6.53, 0.006327, 3],
 ['pMMlJ6x', 5.02,

In [5]:
for i in reverse_super_alpha_list:
    print(i)

['MeZWWm6', -7.49, -9.21, -0.003027]
['Me6N2Ra', -6.6, -7.77, -0.00442]
['8ENZXYa', -5.8, -5.92, -0.003729]
['QNzx3qK', -5.41, -5.79, -0.004264]


## 4.对筛选出的alpha进行prod corr检查

- 修改保存在输入的super_list全局变量中,**支持断点继续**

- 对于none prod corr再获取

In [ ]:
#最后一个参数为开始位置，可断点继续
check_prod(super_list,0)

b'{"user":{"id":"WY79548"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
0
{'NJeaoAe': 0.7736}
['NJeaoAe', 7.39, 0.1116, 10.34, 0.004381, 3, 0.7736]
Processed 0 alphas
{'m0a2grE': 0.8889}
['m0a2grE', 7.36, 0.1273, 9.97, 0.003672, 3, 0.8889]
Processed 1 alphas
{'2X5xvj5': 0.7791}
['2X5xvj5', 7.1, 0.0835, 8.45, 0.004242, 3, 0.7791]
Processed 2 alphas
{'ggeGlgJ': 0.7954}
['ggeGlgJ', 7.0, 0.0835, 8.35, 0.004259, 3, 0.7954]
Processed 3 alphas
{'6od52QK': 0.8396}
['6od52QK', 5.59, 0.0689, 7.11, 0.005883, 3, 0.8396]
Processed 4 alphas
{'br6EkqZ': 0.8396}
['br6EkqZ', 5.59, 0.0689, 7.11, 0.005883, 3, 0.8396]
Processed 5 alphas
{'aQGRdl9': 0.7719}
['aQGRdl9', 6.93, 0.0835, 8.19, 0.004179, 3, 0.7719]
Processed 6 alphas
{'qYGRlpE': 0.7822}
['qYGRlpE', 6.89, 0.0835, 8.16, 0.004201, 3, 0.7822]
Processed 7 alphas
{'Lmm13LM': 0.7705}
['

In [38]:
cnt_not_none=0
cnt_len=0
cccnt=0
jishu=[]
none_list=[]
for i in super_list:
    cccnt+=1
    if(len(i)>=7):
        cnt_len+=1
        if(i[6] is not None):
            cnt_not_none+=1
        else:
            jishu.append(cccnt-1)
            none_list.append(i)
    else:
         jishu.append(cccnt-1)
         none_list.append(i)
print(f"Total alphas with length >= 7: {cnt_len}, Alphas with non-None prod check: {cnt_not_none}")





Total alphas with length >= 7: 265, Alphas with non-None prod check: 255


In [39]:

result_bag=check_prod(none_list,0)

b'{"user":{"id":"WY79548"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
0
{'9d8j032': 0.7584}
['9d8j032', 6.74, 0.0957, 7.92, 0.003611, 3, 0.7584]
Processed 0 alphas
{'m0ojEo6': 0.8138}
['m0ojEo6', 7.09, 0.1417, 8.93, 0.003176, 3, 0.8138]
Processed 1 alphas
{'E88w0oR': 0.8398}
['E88w0oR', 8.41, 0.1448, 9.44, 0.002521, 3, 0.8398]
Processed 2 alphas
{'Xve3kKa': 0.811}
['Xve3kKa', 6.63, 0.0795, 7.3, 0.00381, 3, 0.811]
Processed 3 alphas
{'1lGlZlK': 0.8071}
['1lGlZlK', 6.45, 0.0798, 7.23, 0.003934, 3, 0.8071]
Processed 4 alphas
{'E8QMdb9': 0.7759}
['E8QMdb9', 6.64, 0.0818, 7.32, 0.003714, 3, 0.7759]
Processed 5 alphas
{'rNRNPJE': 0.7759}
['rNRNPJE', 6.64, 0.0818, 7.32, 0.003714, 3, 0.7759]
Processed 6 alphas
{'Rl5z7aj': 0.7937}
['Rl5z7aj', 6.54, 0.0801, 7.22, 0.003811, 3, 0.7937]
Processed 7 alphas
{'qY7Jk3K': 0.7471}
['qY7

In [44]:
for i in range(len(none_list)):
    print(f"none_list[{i}]: {none_list[i][0]},super_list[{jishu[i]}]: {super_list[jishu[i]][0]}")
    if(none_list[i][0]==super_list[jishu[i]][0]):
        if(len(none_list[i]) >= 8):
            none_list[i][6]=none_list[i][7]
        super_list[jishu[i]]=none_list[i]

none_list

none_list[0]: 9d8j032,super_list[42]: 9d8j032
none_list[1]: m0ojEo6,super_list[43]: m0ojEo6
none_list[2]: E88w0oR,super_list[44]: E88w0oR
none_list[3]: Xve3kKa,super_list[45]: Xve3kKa
none_list[4]: 1lGlZlK,super_list[46]: 1lGlZlK
none_list[5]: E8QMdb9,super_list[47]: E8QMdb9
none_list[6]: rNRNPJE,super_list[48]: rNRNPJE
none_list[7]: Rl5z7aj,super_list[49]: Rl5z7aj
none_list[8]: qY7Jk3K,super_list[121]: qY7Jk3K
none_list[9]: d3YpE3X,super_list[122]: d3YpE3X
none_list[10]: RlPQ2Na,super_list[123]: RlPQ2Na
none_list[11]: qYOAX1V,super_list[167]: qYOAX1V
none_list[12]: oJE2pxl,super_list[168]: oJE2pxl
none_list[13]: OMRERO1,super_list[169]: OMRERO1
none_list[14]: 0Nx7Obv,super_list[195]: 0Nx7Obv
none_list[15]: l5aqmx2,super_list[204]: l5aqmx2
none_list[16]: z2g65KK,super_list[205]: z2g65KK
none_list[17]: eVNA2Ap,super_list[206]: eVNA2Ap


[['9d8j032', 6.74, 0.0957, 7.92, 0.003611, 3, 0.7584],
 ['m0ojEo6', 7.09, 0.1417, 8.93, 0.003176, 3, 0.8138],
 ['E88w0oR', 8.41, 0.1448, 9.44, 0.002521, 3, 0.8398],
 ['Xve3kKa', 6.63, 0.0795, 7.3, 0.00381, 3, 0.811],
 ['1lGlZlK', 6.45, 0.0798, 7.23, 0.003934, 3, 0.8071],
 ['E8QMdb9', 6.64, 0.0818, 7.32, 0.003714, 3, 0.7759],
 ['rNRNPJE', 6.64, 0.0818, 7.32, 0.003714, 3, 0.7759],
 ['Rl5z7aj', 6.54, 0.0801, 7.22, 0.003811, 3, 0.7937],
 ['qY7Jk3K', 6.0, 0.0756, 6.21, 0.003549, 3, 0.7471, 0.7471, 0.7471],
 ['d3YpE3X', 6.22, 0.1355, 7.6, 0.002984, 3, 0.8613, 0.8613, 0.8613],
 ['RlPQ2Na', 5.89, 0.0941, 6.68, 0.00342, 3, 0.5356, 0.5356, 0.5356],
 ['qYOAX1V', 6.31, 0.1343, 7.13, 0.002556, 3, 0.7185, 0.7185, 0.7185],
 ['oJE2pxl', 6.5, 0.0855, 6.23, 0.002688, 3, 0.7494, 0.7494, 0.7494],
 ['OMRERO1', 7.06, 0.1438, 7.43, 0.002216, 3, 0.6974, 0.6974, 0.6974],
 ['0Nx7Obv', 5.24, 0.1173, 6.26, 0.003042, 3, 0.6523, 0.6523, 0.6523],
 ['l5aqmx2', 5.21, 0.1094, 6.05, 0.003085, 3, 0.8073, 0.8073, 0.8073],

## 5.排序输出


In [46]:
super_list.sort(key=super_score, reverse=True)
super_list 

[['qYVemlO', 5.44, 0.1071, 5.15, 0.002092, 3, 0.3812],
 ['K5ZRRbg', 6.77, 0.147, 6.28, 0.001723, 3, 0.4346],
 ['YEYqXqv', 5.66, 0.1176, 5.82, 0.002245, 3, 0.4733],
 ['eVEwoJN', 5.92, 0.1136, 6.58, 0.002718, 3, 0.4754],
 ['1lPxLaW', 6.52, 0.1504, 5.78, 0.001569, 3, 0.4869],
 ['Vm57PL8', 6.38, 0.1326, 5.99, 0.00176, 3, 0.4985],
 ['pM3gkwq', 5.48, 0.1168, 5.27, 0.001981, 3, 0.4998],
 ['MeVWY09', 6.0, 0.111, 6.91, 0.002987, 3, 0.5318],
 ['RlPQ2Na', 5.89, 0.0941, 6.68, 0.00342, 3, 0.5356, 0.5356, 0.5356],
 ['1lPWq3z', 7.2, 0.1645, 6.49, 0.001623, 3, 0.5357],
 ['z2e1aQE', 5.97, 0.1108, 6.88, 0.003001, 3, 0.5405],
 ['OMgqJ1R', 5.53, 0.1099, 6.13, 0.002799, 3, 0.5419],
 ['z20ArAo', 6.55, 0.1316, 6.24, 0.001818, 3, 0.5433],
 ['Wo3YlZd', 6.17, 0.0876, 5.58, 0.002335, 3, 0.5583],
 ['eVv0o2O', 6.54, 0.1259, 7.43, 0.002581, 3, 0.5721],
 ['d3om1mE', 6.22, 0.1461, 6.19, 0.001978, 3, 0.5735],
 ['m01mgm9', 5.52, 0.1154, 5.89, 0.002467, 3, 0.5788],
 ['RlKAV5o', 5.56, 0.1143, 5.94, 0.002496, 3, 0.5812],


In [47]:
flag= True
for item in super_list:
    print(item)
    if (item[-1]>0.7)&flag:
        flag=False
        print("-------------------除了正上方那一个以上均为prod<0.7------------------------")


['qYVemlO', 5.44, 0.1071, 5.15, 0.002092, 3, 0.3812]
['K5ZRRbg', 6.77, 0.147, 6.28, 0.001723, 3, 0.4346]
['YEYqXqv', 5.66, 0.1176, 5.82, 0.002245, 3, 0.4733]
['eVEwoJN', 5.92, 0.1136, 6.58, 0.002718, 3, 0.4754]
['1lPxLaW', 6.52, 0.1504, 5.78, 0.001569, 3, 0.4869]
['Vm57PL8', 6.38, 0.1326, 5.99, 0.00176, 3, 0.4985]
['pM3gkwq', 5.48, 0.1168, 5.27, 0.001981, 3, 0.4998]
['MeVWY09', 6.0, 0.111, 6.91, 0.002987, 3, 0.5318]
['RlPQ2Na', 5.89, 0.0941, 6.68, 0.00342, 3, 0.5356, 0.5356, 0.5356]
['1lPWq3z', 7.2, 0.1645, 6.49, 0.001623, 3, 0.5357]
['z2e1aQE', 5.97, 0.1108, 6.88, 0.003001, 3, 0.5405]
['OMgqJ1R', 5.53, 0.1099, 6.13, 0.002799, 3, 0.5419]
['z20ArAo', 6.55, 0.1316, 6.24, 0.001818, 3, 0.5433]
['Wo3YlZd', 6.17, 0.0876, 5.58, 0.002335, 3, 0.5583]
['eVv0o2O', 6.54, 0.1259, 7.43, 0.002581, 3, 0.5721]
['d3om1mE', 6.22, 0.1461, 6.19, 0.001978, 3, 0.5735]
['m01mgm9', 5.52, 0.1154, 5.89, 0.002467, 3, 0.5788]
['RlKAV5o', 5.56, 0.1143, 5.94, 0.002496, 3, 0.5812]
['vp6lmMr', 6.39, 0.1444, 6.74, 0.00